# Modified rough Bergomi graphics generation 

Jupyter Notebook to generate the graphics needed in paper:

- C. Bayer, P.K. Friz, A. Gulisashvili, B. Horvath, B. Stemper (2017). *Short dated option pricing under rough volatility.*



In [1]:
# Standard library imports
from math import sqrt, log, e
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline 

# Import RoughStochVol Package
import code.BlackScholes as BS
import code.ModRoughBergomi as MRB

Parameter setting

In [2]:
# Manual creation of Simulation ID (for creating log-files)
sim_id = 9

# Seed setting for reproducibility of stochastic simulations
np.random.seed(0)

# Model parameters
spot_price = 1
spot_variance = 0.0654
vol_of_vol = 0.2928
rho = -0.7571
hurst_index = 0.1

# Monte Carlo parameters
time_steps = 500
mc_runs = 10**5

# Option parameters
flag = 'c'
last_time = 1
maturities_list = np.linspace(last_time/time_steps, last_time, time_steps)
log_strike_const = 0.4

# Asymptotics parameters
beta = 0.06
spot_vol = sqrt(spot_variance)
spot_vol_prime = 1/2 * vol_of_vol * spot_vol

In [3]:
# Writing parameters to log-file
with open("%s_log.txt" %sim_id, "w") as log_file:
    log_file.write("# Parameters # \n")
    log_file.write("Simulation ID: %s \n" % sim_id)
    log_file.write("Spot price: %s \n" % spot_price)
    log_file.write("Spot variance: %s \n" % spot_variance)
    log_file.write("Vol of vol: %s \n" % vol_of_vol)
    log_file.write("Rho: %s \n" % rho)
    log_file.write("Hurst index: %s \n" % hurst_index)
    log_file.write("Time steps: %s \n" % time_steps)
    log_file.write("MC runs (individual batches): %s \n" % mc_runs)
    log_file.write("Flag: %s \n" % flag)
    log_file.write("Last time: %s \n" % last_time)
    log_file.write("Log strike constant: %s \n" % log_strike_const)
    log_file.write("Beta: %s \n" % beta)

# Data generation

Section to generate data for plots:
- one cell to generate asymptotics data (log strikes, abs log call prices, implied vols)
- one cell to do Cholesky Monte Carlo generation of option prices (single run)
- one cell to do Cholesky Monte Carlo generation of very large samples sizes via splitting up in batches, saving intermediate results and later collecting and merging individual results

### Asymptotics:

In [4]:
# INITIALIZATION
log_strikes = np.zeros(maturities_list.size)
asy_abs_log_call_price = np.zeros(maturities_list.size)
asy_imp_vol = np.zeros(maturities_list.size)

# GENERATION
for index, item in enumerate(maturities_list):

    asy = MRB.Asymptotics(hurst_index, log_strike_const, item, beta, rho, 
                          spot_vol, spot_vol_prime)

    # Computing and storing asymptotics
    log_strikes[index] = asy.get_log_strike()
    asy_abs_log_call_price[index] = asy.get_abs_log_call_price()
    asy_imp_vol[index] = asy.get_implied_vol()

### Rough vol Cholesky Monte Carlo (Single run)

In [ ]:
# INITIALIZATION
# ==============
strikes = np.exp(log_strikes)

mc = CholeskyMC(hurst_index, time_steps, last_time, rho, 
                spot_variance, spot_price, vol_of_vol)

# DATA GENERATION
# ===============

# Compute asset price paths
log_price_paths = mc.get_log_price_paths(mc_runs)[:, 1:]
price_paths = np.exp(log_price_paths)

# Compute option prices
option_prices = np.maximum(price_paths - strikes, 0)
chol_option_prices = np.average(option_prices, axis=0)
chol_option_prices_std = np.sqrt(np.var(option_prices, axis=0)/mc_runs)

*Saving and loading of data*

In [ ]:
np.save("%s_chol_option_prices" %sim_id, chol_option_prices)
np.save("%s_chol_option_prices_std", chol_option_prices_std)

In [ ]:
chol_option_prices = np.load("1_chol_option_prices.npy")
chol_option_prices_std = np.load("1_chol_option_prices_std.npy")

*Transfer to absolute log call price*

In [ ]:
chol_abs_log_call_price = np.abs(np.log(chol_option_prices))

### Rough vol Cholesky Monte Carlo (Batch runs)

In [ ]:
# INITIALIZATION
# ==============
nb_batches = 1000
strikes = np.exp(log_strikes)
sums_avgs = np.zeros(time_steps)
sums_vars = np.zeros(time_steps)

# Initialisation of object creates Cholesky matrix 
mc = MRB.CholeskyPricer(hurst_index, time_steps, last_time, rho, 
                        spot_variance, spot_price, vol_of_vol)

# BATCH COMPUTATION
# =================

for index in range(nb_batches):
    
    # Compute asset price paths
    log_price_paths = mc.get_log_price_paths(mc_runs)[:, 1:]
    price_paths = np.exp(log_price_paths)

    # Convert to option prices
    option_prices = np.maximum(price_paths - strikes, 0)
    
    # Add to running sums
    sums_avgs += np.average(option_prices, axis=0)
    sums_vars += np.var(option_prices, axis=0)
    
    print("run %i completed" %index)

# PUTTING BATCHES TOGETHER
chol_option_prices = sums_avgs/nb_batches
chol_option_prices_std = np.sqrt(sums_vars/(nb_batches*mc_runs))

# ADDING INFO TO LOGFILE
with open("%s_log.txt" %sim_id, "a") as log_file:
    log_file.write("Number of batches (each with MC runs): %s \n" % nb_batches )

*Saving and loading of data*

In [ ]:
np.save("%s_chol_option_prices" % sim_id, chol_option_prices)
np.save("%s_chol_option_prices_std" % sim_id, chol_option_prices_std)

In [ ]:
chol_option_prices = np.load("1_chol_option_prices.npy")
chol_option_prices_std = np.load("1_chol_option_prices_std.npy")

*Confidence interval computations*

In [ ]:
# 95% Confidence interval bounds for calls
chol_call_ci_top = chol_option_prices + 1.96 * chol_option_prices_std
chol_call_ci_low = chol_option_prices - 1.96 * chol_option_prices_std

# 95% Confidence interval bounds for log call (note inversion)
chol_logcall_ci_low = np.abs(np.log(chol_call_ci_top))
chol_logcall_ci_top = np.abs(np.log(chol_call_ci_low))

*Transfer to absolute log call price*

In [ ]:
chol_abs_log_call_price = np.abs(np.log(chol_option_prices))

### Black-Scholes

In [ ]:
# INITIALIZATION
strikes = np.exp(log_strikes)
bsgbm_option_prices = np.zeros(maturities_list.size)
bsgbm_option_prices_std = np.zeros(maturities_list.size)
mc_samples = 10**8

# DATA GENERATION
for index, time in enumerate(maturities_list):

    bs = BS.Pricer(flag, spot_price, strikes[index], time, spot_vol)

    bsgbm_option_prices[index], bsgbm_option_prices_std[index] = bs.get_price_via_GBM(mc_samples)
    
# LOGGING
with open("%s_log.txt" %sim_id, "a") as log_file:
    log_file.write("BS GBM Monte Carlo Samples: %s \n" % mc_samples )

*Saving and loading of data*

In [ ]:
np.save("%s_bsgbm_option_prices" % sim_id, bsgbm_option_prices)
np.save("%s_bsgbm_option_prices_std" % sim_id, bsgbm_option_prices_std)

In [ ]:
bsgbm_option_prices = np.load("1_bsgbm_option_prices.npy")
bsgbm_option_prices_std = np.load("1_bsgbm_option_prices_std.npy")

*Computation of confidence intervals*

In [ ]:
# 95% Confidence interval bounds for calls
bsgbm_call_ci_top = bsgbm_option_prices + 1.96 * bsgbm_option_prices_std
bsgbm_call_ci_low = bsgbm_option_prices - 1.96 * bsgbm_option_prices_std

# 95% Confidence interval bounds for log call (note inversion)
bsgbm_logcall_ci_low = np.abs(np.log(bsgbm_call_ci_top))
bsgbm_logcall_ci_top = np.abs(np.log(bsgbm_call_ci_low))

*Transfer to absolute log call price*

In [ ]:
bsgbm_abs_log_call_price = np.abs(np.log(bsgbm_option_prices))

### Test cells

Generate Log Call Prices via BS Formula from Asymptotic Implied Vol Formula

In [ ]:
# calls_via_asyimpvol = []
# strikes = np.exp(log_strikes)

# for index, item in enumerate(maturities_list):
    
#     BS = BlackScholes(flag, spot_price, strikes[index], item, asy_imp_vol[index])
    
#     calls_via_asyimpvol.append(BS.get_option_price())
    
# abs_logcalls_via_asyimpvol = abs(np.log(calls_via_asyimpvol))

# Plotting #

In [ ]:
import seaborn as sns

sns.set_context("paper")
sns.set(font='serif')
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]})

## Absolute log call price:

In [ ]:
ratio = chol_abs_log_call_price/asy_abs_log_call_price

In [ ]:
fig2 = plt.figure()
alcp = fig2.add_subplot(1,1,1)

alcp.set_title('Rough vol: Absolute log call price dynamics')
alcp.set_xlabel('Time to maturity $t$')
alcp.set_ylabel('Absolute log call price $|\log \, c(k_t, t)|$')
alcp.plot(maturities_list, asy_abs_log_call_price, '-g', label='Asymptotic formula')
alcp.plot(maturities_list, chol_abs_log_call_price, '--b', label='Cholesky Pricer')
#alcp.plot(maturities_list, bsgbm_abs_log_call_price, '--b', label='Geometric Brownian Motion (GBM)')
alcp.fill_between(maturities_list, chol_logcall_ci_low, chol_logcall_ci_top, interpolate=True, alpha=0.25, 
                facecolor='blue', label='95% Confidence Interval Cholesky')
alcp.set_xlim([0, last_time])
#alcp.set_ylim([0, 50])


alcp2 = alcp.twinx()
alcp2.set_ylabel('Ratio Cholesky/Asymptotics')
alcp2.plot(maturities_list, ratio, '-m', label='Ratio Cholesky/Asymptotics')
alcp.set_xlim([0,last_time])


lines, labels = alcp.get_legend_handles_labels()
lines2, labels2 = alcp2.get_legend_handles_labels()
alcp2.legend(lines + lines2, labels + labels2, loc='upper right', frameon=True)

In [ ]:
fig2.savefig("%s_logcall.pdf" %sim_id, dpi=500)

## Implied volatility:

Computing implied volatilities from Monte Carlo prices:

In [ ]:
# INITIALIZATION
chol_imp_vol = np.zeros(maturities_list.size)
bsgbm_imp_vol = np.zeros(maturities_list.size)
strikes = np.exp(log_strikes)
lower_bound = 0.2
upper_bound = 0.5

# COMPUTATION
for index, item in enumerate(maturities_list):
    
    obj = BS.ImpliedVol(flag, chol_option_prices[index], spot_price, strikes[index], item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    chol_imp_vol[index] = obj.get()

In [ ]:
# INITIALIZATION
iv_upper = np.zeros(maturities_list.size)
iv_lower = np.zeros(maturities_list.size)

# COMPUTATION
for index, item in enumerate(maturities_list):
    
    obj1 = BS.ImpliedVol(flag, chol_call_ci_top[index], spot_price, strikes[index], item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    obj2 = BS.ImpliedVol(flag, chol_call_ci_low[index], spot_price, strikes[index], item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    

    iv_upper[index] = obj1.get()
    iv_lower[index] = obj2.get()

In [ ]:
iv_upper-iv_lower

### Implied vol error

In [ ]:
ratio = chol_imp_vol/asy_imp_vol

### Plotting

In [ ]:
fig3 = plt.figure()
iv = fig3.add_subplot(1,1,1)
iv.set_title('Rough vol: Implied volatility dynamics')
#iv.set_title('Rough vol: Implied volatility dynamics')
iv.set_xlabel('Time to maturity $t$')
iv.set_ylabel('Implied volatility $\sigma_{impl}(k_t, t)$')
iv.plot(maturities_list, asy_imp_vol,'-g', linewidth=1,label='Asymptotic formula')
#iv.plot(maturities_list, bsgbm_imp_vol, '-b', label='Geometric Brownian Motion (GBM)')
iv.plot(maturities_list, chol_imp_vol, 'ob',markersize=0.5, label='Cholesky Pricer')
iv.fill_between(maturities_list, iv_lower, iv_upper, interpolate=True, alpha=0.25, 
                facecolor='blue', label='95% Normal Confidence Interval')
#iv.plot(maturities_list, mc_imp_vol, '-', markersize=3, label='Monte Carlo')
#iv.plot(maturities_list, test, '-', label='Numerical from Asymptotic Log call')
#iv.semilogx(maturities_list, spot_vol, '-', label='Black-Scholes')
iv.set_xlim([0, last_time])
#iv.set_ylim([0.255, 0.2561])


iv2 = iv.twinx()
iv2.plot(maturities_list, ratio, '-m', markersize=3, label='Ratio Cholesky/Asymptotics')
iv2.set_xlim([0, last_time])
iv2.set_ylabel('Ratio Cholesky/Asymptotics')
#iv2.set_ylabel('Relative error in %')
#iv2.plot(maturities_list, rel_err_pc, '+m', markersize=3, label='Relative error')
#iv2.semilogy(maturities_list, abs_err, '+', markersize=3, label='Absolute error')
#iv2.set_ylabel('Absolute error')
iv2.set_xlim([0, last_time])
#iv2.set_ylim([0.995, 1.015])

#iv3 = iv.twinx()
# iv3.plot(maturities_list, abs_err, label='Absolute error')
# Second, show the right spine.
# iv3.spines["right"].set_visible(True)
# iv3.spines["right"].set_position(("axes", 1.15))
# iv3.set_ylabel('Relative error in %')
# iv3.plot(maturities_list, rel_err_pc, '+c', markersize=3, label='Relative error')
#iv3.semilogy(maturities_list, abs_err, '+m', markersize=3, label='Absolute error')
#iv3.set_ylabel('Absolute error')
# iv3.set_xlim([0, last_time])
#iv3.set_ylim([0, 0.0004])


lines, labels = iv.get_legend_handles_labels()
lines2, labels2 = iv2.get_legend_handles_labels()
#lines3, labels3 = iv3.get_legend_handles_labels()
iv.legend(lines + lines2, labels + labels2, loc='upper right', frameon=True)

In [ ]:
del(lines2, labels2)

In [ ]:
fig3.savefig("%s_impvolDetail.pdf" %sim_id, dpi=500)

## Log-strikes

In [ ]:
fig1 = plt.figure()
ls = fig1.add_subplot(1,1,1)
ls.set_title('Log-strike behaviour')
ls.set_xlabel('Time to maturity')
ls.set_ylabel('Log-strike')
ls.plot(maturities_list, log_strikes)

In [ ]:
fig1.savefig("Log_strike4.pdf")

# Pair plot Black-Scholes GBM

In [ ]:
# LOADING DATA

atm_bsgbm_option_prices = np.load("3_bsgbm_option_prices.npy")
atm_bsgbm_option_prices_std = np.load("3_bsgbm_option_prices_std.npy")

motm_bsgbm_option_prices = np.load("1_bsgbm_option_prices.npy")
motm_bsgbm_option_prices_std = np.load("1_bsgbm_option_prices_std.npy")

In [ ]:
# 95% Confidence interval bounds for ATM calls
atm_bsgbm_call_ci_top = atm_bsgbm_option_prices + 1.96 * atm_bsgbm_option_prices_std
atm_bsgbm_call_ci_low = atm_bsgbm_option_prices - 1.96 * atm_bsgbm_option_prices_std

# 95% Confidence interval bounds for MOTM calls
motm_bsgbm_call_ci_top = motm_bsgbm_option_prices + 1.96 * motm_bsgbm_option_prices_std
motm_bsgbm_call_ci_low = motm_bsgbm_option_prices - 1.96 * motm_bsgbm_option_prices_std

*Implied vol computations*

In [ ]:
# INITIALIZATION
# ==============

atm_bsgbm_imp_vol = np.zeros(maturities_list.size)
motm_bsgbm_imp_vol = np.zeros(maturities_list.size)

atm_iv_top = np.zeros(maturities_list.size)
atm_iv_low = np.zeros(maturities_list.size)

motm_iv_top = np.zeros(maturities_list.size)
motm_iv_low = np.zeros(maturities_list.size)

strikes = np.exp(log_strikes)
lower_bound = 0.2
upper_bound = 0.5

# COMPUTATION
# ===========

for index, item in enumerate(maturities_list):
    
    atm_obj = BS.ImpliedVol(flag, atm_bsgbm_option_prices[index], spot_price, 1, item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    atm_obj_top = BS.ImpliedVol(flag, atm_bsgbm_call_ci_top[index], spot_price, 1, item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    atm_obj_low = BS.ImpliedVol(flag, atm_bsgbm_call_ci_low[index], spot_price, 1, item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    motm_obj = BS.ImpliedVol(flag, motm_bsgbm_option_prices[index], spot_price, strikes[index], item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    motm_obj_top = BS.ImpliedVol(flag, motm_bsgbm_call_ci_top[index], spot_price, strikes[index], item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    motm_obj_low = BS.ImpliedVol(flag, motm_bsgbm_call_ci_low[index], spot_price, strikes[index], item, 
                         lower_bound, upper_bound, maxiter=1000, method ='f')
    
    atm_bsgbm_imp_vol[index] = atm_obj.get()
    motm_bsgbm_imp_vol[index] = motm_obj.get()
    
    atm_iv_top[index] = atm_obj_top.get()
    atm_iv_low[index] = atm_obj_low.get()
    
    motm_iv_top[index] = motm_obj_top.get()
    motm_iv_low[index] = motm_obj_low.get()

In [ ]:
fig4 = plt.figure(figsize=(10,6))
fig4.suptitle("Black-Scholes: Implied volatility dynamics")

# LEFT FIGURE
ax1 = fig4.add_subplot(1,2,1)
ax1.set_title('ATM: $k_t=0$')
ax1.set_xlabel('Time to maturity $t$ (in years)')
ax1.set_ylabel('Implied volatility $\sigma_{impl}(k_t, t)$')
ax2.set_xlim([0, 0.5])
ax1.set_ylim([0.2552, 0.2563])
ax1.plot(maturities_list, asy_imp_vol, '-g', label='Asymptotic formula')
ax1.plot(maturities_list, atm_bsgbm_imp_vol, '-b', linewidth=1,label='Geometric Brownian Motion (GBM)')
ax1.fill_between(maturities_list, atm_iv_top, atm_iv_low, interpolate=True, alpha=0.25, 
                facecolor='blue', label='95% Normal Confidence Interval')

# RIGHT FIGURE
ax2 = fig4.add_subplot(1,2,2)
ax2.set_title('MOTM: $k_t = 0.4t^{0.3}$')
ax2.set_xlabel('Time to maturity $t$ (in years)')
#ax2.set_ylabel('Implied volatility $\sigma_{impl}(k_t, t)$')
ax2.set_xlim([0, 0.5])
ax2.set_ylim([0.2552, 0.2563])
ax2.plot(maturities_list, asy_imp_vol, '-g', label='Asymptotic formula')
ax2.plot(maturities_list, motm_bsgbm_imp_vol, '-b', linewidth=1,label='Geometric Brownian Motion (GBM)')
ax2.fill_between(maturities_list, motm_iv_top, motm_iv_low, interpolate=True, alpha=0.25, 
                facecolor='blue', label='95% Normal Confidence Interval')

lines, labels = ax1.get_legend_handles_labels()
#fig4.legend(lines, labels, loc = 'lower center', ncol=1, frameon=True )
fig4.legend( lines, labels, loc = 'lower center', bbox_to_anchor = (0,-0.1,1,1),
            bbox_transform = plt.gcf().transFigure, ncol=3, frameon=True )
#fig4.tight_layout()

In [ ]:
fig4.savefig("%s_impvol_pairT05.pdf" %sim_id, bbox_inches='tight', dpi=500)
#fig.savefig('samplefigure', bbox_extra_artists=(lgd,), bbox_inches='tight')

# Quadruple Plot Rough Implied Vol